In [2]:
# Global packages
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, roc_curve, roc_auc_score, auc
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torchvision import models, transforms
from torch.utils.data import DataLoader, Subset
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive


In [4]:
import torchvision.transforms as T
# from albumentations import Compose, HorizontalFlip, VerticalFlip, Rotate, RandomBrightnessContrast, Normalize, Resize
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2

# torchvision augmentation pipeline
def torchvision_augmentation():
    return T.Compose([
        T.RandomHorizontalFlip(p=0.5),
        T.RandomVerticalFlip(p=0.5),
        T.RandomRotation(degrees=45),
        T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# Albumentations augmentation pipeline
def albumentations_augmentation():
    return A.Compose([
        # HorizontalFlip(p=0.5),
        # VerticalFlip(p=0.5),
        # Rotate(limit=45, p=0.5),
        # RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        # Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        # ToTensorV2()
        A.HorizontalFlip(p=0.5),  # Flips the image horizontally with a 50% chance
        A.VerticalFlip(p=0.5),    # Flips the image vertically with a 50% chance
        A.RandomRotate90(p=0.5),  # Randomly rotates the image 90 degrees with a 50% chance
        A.Rotate(limit=40, p=0.5),  # Randomly rotates between -40 to +40 degrees
        A.Blur(blur_limit=3, p=0.5),  # Applies blur
        A.OpticalDistortion(p=0.3),  # Applies optical distortion
        A.GridDistortion(p=0.3),  # Applies grid distortion
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=50, p=0.5),  # Randomly changes hue, saturation, value
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),  # Randomly changes brightness and contrast
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # Adds Gaussian noise
        A.CLAHE(clip_limit=2, p=0.5),  # Applies CLAHE (Contrast Limited Adaptive Histogram Equalization)
        A.ToGray(p=0.2),  # Converts to grayscale with 20% chance
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # Normalizes image
        ToTensorV2()  # Converts the image to a PyTorch tensor
    ])

# Load an image with PIL for torchvision and convert to tensor
def load_image_for_torchvision(path):
    image = Image.open(path)
    return torchvision_augmentation()(image)

# Load an image with OpenCV for Albumentations and apply transformation
def load_image_for_albumentations(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    augmented = albumentations_augmentation()(image=image)
    return augmented['image']

import matplotlib.pyplot as plt
import numpy as np

def plot_and_save_augmented_images(image_path):
    # Load images
    torch_img = load_image_for_torchvision(image_path)
    alb_img = load_image_for_albumentations(image_path)

    # Convert tensor to numpy array for plotting (Torchvision)
    torch_img = torch_img.permute(1, 2, 0).numpy()
    torch_img = np.clip(torch_img, 0, 1)  # Clipping after normalization to avoid display issues

    # Convert tensor to numpy array for plotting (Albumentations)
    alb_img = alb_img.permute(1, 2, 0).numpy()  # This change is necessary for correct plotting
    alb_img = np.clip(alb_img, 0, 1)  # Normalize for proper visualization

    # Plot images
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(torch_img)
    axs[0].set_title('Torchvision Augmented Image')
    axs[0].axis('off')

    axs[1].imshow(alb_img)
    axs[1].set_title('Albumentations Augmented Image')
    axs[1].axis('off')

    # Save the figure
    plt.savefig('augmented_images.png')
    plt.show()

# Example usage with a sample image path
image_paths = ['../Dataset_BUSI_with_GT/benign/benign (1).png',
               '../Dataset_BUSI_with_GT/benign/benign (2).png',
               '../Dataset_BUSI_with_GT/benign/benign (3).png',
               '../Dataset_BUSI_with_GT/benign/benign (4).png',
               '../Dataset_BUSI_with_GT/benign/benign (5).png']
for path in image_paths:
  plot_and_save_augmented_images(path)


Output hidden; open in https://colab.research.google.com to view.